In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  1709k      0  0:00:48  0:00:48 --:--:-- 3568k


In [3]:
!rm -r aclImdb/train/unsup

In [4]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")              # defining base directory
val_dir = base_dir / "val"                      # defining validation directory
train_dir = base_dir / "train"                  # defining train directory
for category in ("neg", "pos"):                 
    os.makedirs(val_dir / category)             
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)

In [1]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size = batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size = batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size = batch_size
)



Found 20000 files belonging to 2 classes.


2022-10-18 21:36:51.557950: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-10-18 21:36:51.558041: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: v2fftb-HP-Pavilion-Gaming-Laptop-15-dk1xxx
2022-10-18 21:36:51.558050: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: v2fftb-HP-Pavilion-Gaming-Laptop-15-dk1xxx
2022-10-18 21:36:51.558265: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.85.2
2022-10-18 21:36:51.558305: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.65.1
2022-10-18 21:36:51.558317: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 515.65.1 does not match DSO version 510.85.2 -- cannot find working devices in this configuration
2022-10-18 21:36:51.560038: 

Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [2]:
for inputs, targets in train_ds:
    print("Input shape is : ", inputs.shape)
    print("Input's dataype is : ", inputs.dtype)
    print("Target's shape is : ", targets.shape)
    print("Target's datatype is : ", targets.dtype)
    print("Input[0] : ", inputs[0])
    print("Targets[0] : ", targets[0])
    break

Input shape is :  (32,)
Input's dataype is :  <dtype: 'string'>
Target's shape is :  (32,)
Target's datatype is :  <dtype: 'int32'>
Input[0] :  tf.Tensor(b"An unpleasant woman and an equally unpleasant man are violently and horribly assaulted by a group of two-dimensional psycho thugs during a night-time encounter on a forest road in Shropshire, England. The man and woman who were assaulted plan and carry out a revenge attack on their attackers...<br /><br />Utterly repellent piece of voyeuristic trash, somehow masquerading as 'thought-provoking' drama, whilst actually coming across as sub-Michael Winner cr*p (you just know that Oliver Reed and Susan George would have been cast had it so easily have been made in the 1970s). What happens to Alice (Gillian Anderson) and Adam (Danny Dyer) is appalling and devastating, yet Dan Reed somehow manages to rub the viewer's nose in every last glob of its sexual nastiness. His camera lingers hungrily on Anderson's naked body both during and after 

### Processing words as a set: The bag-of-words approach

In [3]:
from tensorflow.keras.layers import TextVectorization

text_vectorization = TextVectorization(
    max_tokens=20000,               # Limiting max tokens to 20000
    output_mode="multi_hot",        # Keeping output mode to be multihot binary vector
)

text_only_train_ds = train_ds.map(lambda x, y: x)        # Taxt extracted from file (label plus text)
text_vectorization.adapt(text_only_train_ds)             # Adapting text vactorization 

binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [4]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [5]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens = 20000, hidden_dim = 16):
    inputs = keras.Input(shape = (max_tokens,))
    x = layers.Dense(hidden_dim, activation ='relu')(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                    loss="binary_crossentropy",
                    metrics=["accuracy"])
    return model


In [6]:
model = get_model()
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [7]:
callbacks = [
        keras.callbacks.ModelCheckpoint("binary_1gram.keras",
        save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),
        validation_data=binary_1gram_val_ds.cache(),
        epochs=10,
        callbacks=callbacks)
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 19s 28ms/step - loss: 0.3991 - accuracy: 0.8314 - val_loss: 0.2864 - val_accuracy: 0.8946
Epoch 2/10
625/625 [==============================] - 13s 21ms/step - loss: 0.2645 - accuracy: 0.9022 - val_loss: 0.2795 - val_accuracy: 0.8960
Epoch 3/10
625/625 [==============================] - 11s 18ms/step - loss: 0.2381 - accuracy: 0.9173 - val_loss: 0.2929 - val_accuracy: 0.8944
Epoch 4/10
625/625 [==============================] - 12s 20ms/step - loss: 0.2236 - accuracy: 0.9250 - val_loss: 0.3101 - val_accuracy: 0.8966
Epoch 5/10
625/625 [==============================] - 13s 21ms/step - loss: 0.2098 - accuracy: 0.9316 - val_loss: 0.3205 - val_accuracy: 0.8956
Epoch 6/10
625/625 [==============================] - 14s 23ms/step - loss: 0.2149 - accuracy: 0.9324 - val_loss: 0.3320 - val_accuracy: 0.8916
Epoch 7/10
625/625 [==============================] - 14s 22ms/step - loss: 0.2109 - accuracy: 0.9330 - val_loss: 0.3422 - val_accuracy:

In [8]:
data  = """Like a scene in a movie or a verse in a song, paragraphs are the building blocks of any good piece of writing. Paragraphs lend a natural rhythm to your writing that makes it a joy to read. The question is, how do you handle them successfully? 
Below, we take a close look at what makes up an effective paragraph and explain how to write one that suits your needs. We also cover some advanced tips. But first, let’s start with the fundamentals. """

In [17]:
inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

import tensorflow as tf

raw_text_data = tf.convert_to_tensor([["This is not a good movie. But for senior citizens would like it. New gen will hate it"],])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

61.06 percent positive
